In [ ]:
!pip install diffusers transformers accelerate torch torchvision
!pip install gradio safetensors

# 📚 Libraries Used in This Notebook

This notebook uses several important libraries to run and optimize **Stable Diffusion** in Google Colab.  
Here’s what each one is used for in general, and how it is used in **my code**.

---

## 🔹 1. `diffusers`
Hugging Face’s library dedicated to diffusion models like Stable Diffusion.  
It is used in the ML community for text-to-image, image-to-image, inpainting, and other generative tasks.  
It provides ready-to-use pipelines and schedulers that simplify running large diffusion models.

**In my code:**
- used for `StableDiffusionPipeline` to load and run Stable Diffusion.  
- used for `EulerDiscreteScheduler` to optimize inference speed in Colab.  
- This is the **core library running the model**.

---

## 🔹 2. `transformers`
Hugging Face’s library for natural language processing and multimodal tasks.  
It is widely used for pre-trained models like BERT, GPT, and CLIP, which handle text embeddings, classification, and more.  
In diffusion workflows, it helps transform text prompts into embeddings the model can understand.

**In my code:**
- used for the **CLIP text encoder**, which converts prompts into embeddings.  
- even though not imported directly, `diffusers` relies on it internally.  
- without this library, the model would not understand the input text.

---

## 🔹 3. `accelerate`
Hugging Face’s library for managing hardware acceleration across CPUs, GPUs, or TPUs.  
It is used for distributing models efficiently across devices and simplifying mixed setups (e.g., multiple GPUs).  
It abstracts away complex device placement so that developers can focus on the model instead of CUDA details.

**In my code:**
- used internally by `diffusers` to optimize execution on Colab’s GPU.  
- ensures smooth model loading and inference without me handling CUDA directly.  

---

## 🔹 4. `torch` (PyTorch)
A deep learning framework that powers most modern AI models.  
It is used for defining neural networks, tensor computations, GPU acceleration, and training/inference of models.  
Most generative AI models, including Stable Diffusion, are implemented in PyTorch.

**In my code:**
- used for GPU detection: `torch.cuda.is_available()`.  
- used for memory management: `torch.cuda.empty_cache()`.  
- used for reproducibility: `torch.manual_seed()`.  
- used for faster inference: `torch.autocast()` (mixed precision).  
- all Stable Diffusion model computations run on PyTorch.

---

## 🔹 5. `torchvision`
An extension of PyTorch focused on computer vision.  
It is used for loading image datasets, applying image transformations, and providing pre-trained vision models.  
It often comes bundled with PyTorch in ML projects.

**In my code:**
- used only as a dependency installed with PyTorch.  
- not used directly in this notebook.  

---

## 🔹 6. `gradio`
A Python library for creating user interfaces for machine learning models.  
It is used widely for building quick demos where users can interact with models through buttons, sliders, textboxes, or images.  
This makes complex ML models accessible to non-technical users.

**In my code:**
- used to build the Colab interface:
  - buttons to load models and generate images  
  - textboxes for prompts and negative prompts  
  - sliders for steps, guidance, and resolution  
  - image display for generated results  
- used for `gr.Progress()` to show real-time progress during generation.  

---

## 🔹 7. `safetensors`
A specialized file format for storing model weights.  
It is used because it loads models faster than `.bin` or `.pt` and avoids security risks from Python pickling.  
This format is optimized for large diffusion models.

**In my code:**
- used for loading Stable Diffusion weights with `use_safetensors=True`.  
- used to speed up model loading and save VRAM in Colab.  
- ensures safe and efficient checkpoint handling.


In [ ]:
import os
import gc
import time
import torch
import gradio as gr
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, EulerDiscreteScheduler
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

# 🔧 Other Imports in This Notebook

Besides the main ML libraries, this notebook also imports standard Python utilities and additional tools that support Stable Diffusion.

---

## 🔹 `os`
The built-in Python library for interacting with the operating system.  
It is used for handling file paths, environment variables, and directories.  
This is common in ML projects when saving models, outputs, or configuring GPU settings.

**In my code:**
- used for managing file paths and directories when handling images or checkpoints.  
- sometimes used for setting environment variables (e.g., `os.environ["CUDA_VISIBLE_DEVICES"]`).  

---

## 🔹 `gc`
The Python garbage collection library.  
It is used for manually cleaning up memory that is no longer needed.  
This is important in GPU-heavy tasks like Stable Diffusion, where unused objects can quickly fill VRAM.

**In my code:**
- used for calling `gc.collect()` after generating images.  
- helps free up CPU memory and prevent crashes in Colab.  

---

## 🔹 `time`
The built-in Python library for handling time-related tasks.  
It is used in ML workflows for logging execution time, creating delays, or measuring performance.

**In my code:**
- used for tracking how long it takes to generate an image.  
- useful for performance optimization and monitoring generation speed.  

---

## 🔹 `StableDiffusionPipeline`, `DPMSolverMultistepScheduler`, `EulerDiscreteScheduler`
These come from the `diffusers` library.  
They are used for running Stable Diffusion pipelines and choosing different schedulers (the mathematical methods that control how the image denoising process progresses).

**In my code:**
- used for `StableDiffusionPipeline` → loads and runs the Stable Diffusion model.  
- used for `EulerDiscreteScheduler` → provides a fast and memory-efficient denoising scheduler (great for Colab).  
- used for `DPMSolverMultistepScheduler` → another scheduler option that can improve image quality at fewer steps.  

---

## 🔹 `PIL.Image`
Part of the Python Imaging Library (PIL), now maintained as **Pillow**.  
It is used for opening, editing, and saving images in Python.  
This is a standard tool in computer vision pipelines.

**In my code:**
- used for displaying generated Stable Diffusion outputs.  
- used for saving results as `.png` or `.jpg` files.  
- sometimes used for resizing or converting image formats.


In [ ]:
class StableDiffusionGenerator:
    def __init__(self):
        self.pipe = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.progress_callback = None
        self.start_time = None
        self.total_steps = 0
        print(f"Using device: {self.device}")

    def progress_fn(self, step, timestep, latents):
        """Callback function to track generation progress"""
        if self.progress_callback and self.start_time:
            elapsed_time = time.time() - self.start_time
            progress_percent = (step / self.total_steps) * 100

            if step > 0:  # Avoid division by zero
                estimated_total_time = elapsed_time * self.total_steps / step
                remaining_time = estimated_total_time - elapsed_time
            else:
                remaining_time = 0

            # Format time nicely
            def format_time(seconds):
                if seconds < 60:
                    return f"{seconds:.1f}s"
                elif seconds < 3600:
                    return f"{seconds//60:.0f}m {seconds%60:.0f}s"
                else:
                    return f"{seconds//3600:.0f}h {(seconds%3600)//60:.0f}m"

            progress_text = f"🎨 Generating: {progress_percent:.1f}% complete | "
            progress_text += f"⏱️ Elapsed: {format_time(elapsed_time)} | "
            progress_text += f"⏳ Remaining: {format_time(remaining_time)}"

            # Update progress through callback
            self.progress_callback(progress_text)

    def load_model(self, model_id="runwayml/stable-diffusion-v1-5", progress=gr.Progress()):
        """Load the Stable Diffusion model with memory optimizations and progress tracking"""
        try:
            # Progress tracking for model loading
            progress(0.1, desc="🔄 Clearing previous models...")

            # Clear any existing model from memory
            if self.pipe is not None:
                del self.pipe
                torch.cuda.empty_cache()
                gc.collect()

            progress(0.3, desc="📥 Downloading model (this may take a while for first time)...")
            print(f"Loading model: {model_id}")

            # Load model with memory optimizations
            self.pipe = StableDiffusionPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                use_safetensors=True,
                variant="fp16" if self.device == "cuda" else None,
                safety_checker=None,  # Disable safety checker to save memory
                requires_safety_checker=False
            )

            progress(0.7, desc="⚙️ Optimizing model for Colab...")

            # Use faster, more memory-efficient scheduler
            self.pipe.scheduler = EulerDiscreteScheduler.from_config(
                self.pipe.scheduler.config
            )

            # Enable all memory optimizations
            if self.device == "cuda":
                self.pipe.enable_attention_slicing(1)  # More aggressive slicing
                self.pipe.enable_vae_slicing()
                self.pipe.enable_vae_tiling()  # Additional VAE optimization

            progress(0.9, desc="🚀 Moving model to device...")
            self.pipe = self.pipe.to(self.device)

            # Enable CPU offloading to save VRAM
            if self.device == "cuda":
                self.pipe.enable_model_cpu_offload()

            progress(1.0, desc="✅ Model loaded successfully!")

            # Clear cache after loading
            if self.device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            print("Model loaded successfully!")
            return "✅ Model loaded successfully! Ready to generate images."

        except Exception as e:
            error_msg = f"❌ Error loading model: {str(e)}"
            print(error_msg)
            return error_msg

    def generate_image(self, prompt, negative_prompt="", num_inference_steps=15,
                      guidance_scale=7.5, width=512, height=512, seed=-1,
                      progress=gr.Progress()):
        """Generate image from text prompt with real-time progress"""

        if self.pipe is None:
            return None, "❌ Please load a model first!"

        try:
            # Validate inputs for optimal Colab performance
            if width > 768 or height > 768:
                return None, "❌ Image dimensions too large for Colab! Keep width and height ≤ 768px"

            if num_inference_steps > 30:
                return None, "❌ Too many inference steps! Keep steps ≤ 30 for faster generation"

            # Set up progress tracking
            self.total_steps = num_inference_steps
            self.start_time = time.time()

            # Progress callback for Gradio
            def update_progress(text):
                progress(None, desc=text)

            self.progress_callback = update_progress

            progress(0.05, desc="🎲 Setting up generation parameters...")

            # Set random seed if specified
            if seed != -1:
                torch.manual_seed(seed)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed(seed)

            progress(0.1, desc="🧹 Clearing memory cache...")

            # Clear cache before generation
            if self.device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            progress(0.15, desc=f"🎨 Starting image generation ({num_inference_steps} steps)...")
            print(f"Generating image for prompt: '{prompt}'")

            # Generate image with progress callback
            with torch.autocast(self.device):
                result = self.pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    width=width,
                    height=height,
                    callback=self.progress_fn,
                    callback_steps=1  # Update progress every step
                )

            image = result.images[0]

            progress(0.95, desc="🖼️ Finalizing image...")

            # Clear cache after generation
            if self.device == "cuda":
                torch.cuda.empty_cache()
            gc.collect()

            total_time = time.time() - self.start_time
            status_msg = f"✅ Image generated successfully in {total_time:.1f}s!"
            progress(1.0, desc=status_msg)
            print(status_msg)
            return image, status_msg

        except Exception as e:
            error_msg = f"❌ Error generating image: {str(e)}"
            print(error_msg)
            progress(1.0, desc=error_msg)
            return None, error_msg

    def get_memory_info(self):
        """Get current memory usage information"""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1024**3
            reserved = torch.cuda.memory_reserved() / 1024**3
            total = torch.cuda.get_device_properties(0).total_memory / 1024**3
            return f"GPU Memory - Used: {allocated:.2f}GB | Reserved: {reserved:.2f}GB | Total: {total:.1f}GB"
        else:
            return "Running on CPU"

# 🖼️ `StableDiffusionGenerator` Class Explained

This class is a **wrapper around the Stable Diffusion pipeline** that makes it easy to:

- load and optimize a model for Colab,
- generate images from text prompts,
- track real-time progress,
- manage GPU memory usage.

It combines utilities from `diffusers`, `torch`, and `gradio` into one organized interface.

---

## 🔹 `__init__`
**Purpose:** Class initializer (constructor).  
- Sets up the device (`cuda` if available, else CPU).  
- Initializes placeholders for the pipeline (`self.pipe`) and progress tracking variables.  

**In this code:**
- `self.pipe = None` → pipeline not yet loaded.  
- `self.device` → auto-detects GPU (important for Colab).  
- `self.progress_callback` → stores a function to update progress in the UI.  
- `self.start_time` + `self.total_steps` → used to estimate how long image generation will take.  
- Prints which device is being used.  

---

## 🔹 `progress_fn(self, step, timestep, latents)`
**Purpose:** A callback function that tracks progress while the model generates images.  
- Called every generation step by the diffusion pipeline.  
- Calculates elapsed time, estimated remaining time, and % complete.  
- Formats times nicely into seconds, minutes, or hours.  
- Sends updates to the Gradio UI via `self.progress_callback`.

**In this code:**
- Gives real-time feedback like:  
  `"🎨 Generating: 46.7% complete | ⏱️ Elapsed: 12.3s | ⏳ Remaining: 14.5s"`  

---

## 🔹 `load_model(self, model_id=..., progress=...)`
**Purpose:** Load the Stable Diffusion model with Colab optimizations.  
- Clears old models from memory (to prevent VRAM leaks).  
- Downloads the specified model from Hugging Face (`from_pretrained`).  
- Configures precision (FP16 on GPU, FP32 on CPU).  
- Disables the safety checker (saves VRAM).  
- Switches scheduler to `EulerDiscreteScheduler` (fast & efficient).  
- Enables optimizations: attention slicing, VAE slicing/tiling, CPU offload.  
- Moves model to GPU/CPU.  
- Returns a success or error message.

**In this code:**
- Default model: `"runwayml/stable-diffusion-v1-5"` (fast & stable).  
- Progress messages update in Gradio during each stage of loading.  

---

## 🔹 `generate_image(self, prompt, negative_prompt="", ...)`
**Purpose:** Generate an image from a text prompt.  
- Validates input (keeps resolution ≤ 768px, steps ≤ 30).  
- Sets up progress tracking (start time, total steps).  
- Applies seed if specified (for reproducibility).  
- Clears GPU memory before generation.  
- Calls `self.pipe(...)` to run Stable Diffusion with the prompt.  
- Uses `self.progress_fn` to update status every step.  
- Collects the output image, clears cache again.  
- Returns `(image, status_message)`.

**In this code:**
- Default settings:  
  - 15 steps,  
  - 512x512 resolution,  
  - guidance scale 7.5 (how strongly the image follows the prompt).  
- Example message after success:  
  `"✅ Image generated successfully in 23.4s!"`  

---

## 🔹 `get_memory_info(self)`
**Purpose:** Report current GPU memory usage.  
- Uses PyTorch APIs (`torch.cuda.memory_allocated`, `torch.cuda.memory_reserved`).  
- Reports how much GPU memory is used, reserved, and the total available.  
- If running on CPU, just returns `"Running on CPU"`.  

**In this code:**
- Helpful for monitoring VRAM in Colab.  
- Example: `"GPU Memory - Used: 2.45GB | Reserved: 3.10GB | Total: 15.8GB"`  

---

# ✅ Summary
This class is a **Colab-optimized Stable Diffusion manager**:
- `__init__` → sets up device and variables.  
- `progress_fn` → tracks real-time progress.  
- `load_model` → loads and optimizes a model.  
- `generate_image` → generates images with progress feedback.  
- `get_memory_info` → reports GPU/CPU memory usage.  


In [ ]:
# Initialize the generator
generator = StableDiffusionGenerator()

## 🔹 `generator = StableDiffusionGenerator()`

**Purpose:**  
This line **creates an instance of my `StableDiffusionGenerator` class**, which I defined earlier.  
It’s like turning the blueprint (class) into a usable object that can load models, generate images, and report memory usage.

---

**What happens when this runs:**
- Calls the `__init__` method of the class automatically.  
- Sets up:
  - `self.pipe = None` → pipeline not loaded yet.  
  - `self.device` → checks if CUDA GPU is available, otherwise falls back to CPU.  
  - `self.progress_callback`, `self.start_time`, `self.total_steps` → initialized to manage progress tracking.  
- Prints out which device is being used:  
  - Example: `"Using device: cuda"` in Colab with GPU.  
  - Example: `"Using device: cpu"` if no GPU available.  

---

**In my code:**  
- `generator` is the **main object** I interact with in later cells.  
- I call:  
  - `generator.load_model(...)` → to load Stable Diffusion.  
  - `generator.generate_image(...)` → to generate images.  
  - `generator.get_memory_info()` → to check GPU memory usage.  
- Without this line, none of the class methods could be used.

In [ ]:
# Define the Gradio interface
def create_interface():
    with gr.Blocks(title="Fast Stable Diffusion Generator", theme=gr.themes.Soft()) as interface:

        gr.Markdown("# ⚡ Fast Stable Diffusion Text-to-Image Generator")
        gr.Markdown("**Optimized for Google Colab with Real-time Progress Tracking**")

        with gr.Tab("🎨 Image Generation"):
            with gr.Row():
                with gr.Column(scale=1):
                    # Model loading section
                    gr.Markdown("### 1. Load Model")
                    with gr.Row():
                        model_dropdown = gr.Dropdown(
                            choices=[
                                "runwayml/stable-diffusion-v1-5",  # Fastest, most stable
                                "stabilityai/stable-diffusion-2-1-base",  # Good balance
                                "CompVis/stable-diffusion-v1-4"  # Alternative option
                            ],
                            value="runwayml/stable-diffusion-v1-5",
                            label="Select Model (v1-5 is fastest)"
                        )
                        load_btn = gr.Button("🚀 Load Model", variant="primary", size="sm")

                    load_status = gr.Textbox(label="Model Status", interactive=False)

                    gr.Markdown("### 2. Generation Settings")
                    prompt = gr.Textbox(
                        label="✨ Prompt",
                        placeholder="a beautiful sunset over mountains, highly detailed, cinematic lighting",
                        lines=3
                    )
                    negative_prompt = gr.Textbox(
                        label="🚫 Negative Prompt (Optional)",
                        placeholder="blurry, bad quality, distorted, ugly",
                        lines=2
                    )

                    with gr.Row():
                        steps = gr.Slider(
                            minimum=8, maximum=30, value=15, step=1,
                            label="⚡ Inference Steps (15 recommended)",
                            info="Fewer steps = faster generation"
                        )
                        guidance = gr.Slider(
                            minimum=3, maximum=15, value=7.5, step=0.5,
                            label="🎯 Guidance Scale",
                            info="Higher = follows prompt more closely"
                        )

                    with gr.Row():
                        width = gr.Dropdown(
                            choices=[256, 384, 512, 640, 768],
                            value=512,
                            label="📐 Width (px)"
                        )
                        height = gr.Dropdown(
                            choices=[256, 384, 512, 640, 768],
                            value=512,
                            label="📐 Height (px)"
                        )

                    seed = gr.Number(
                        label="🎲 Seed (-1 for random)",
                        value=-1,
                        precision=0,
                        info="Same seed = same image"
                    )

                    generate_btn = gr.Button(
                        "🎨 Generate Image",
                        variant="primary",
                        size="lg"
                    )

                    # Quick presets
                    gr.Markdown("### 🚀 Quick Presets")
                    with gr.Row():
                        fast_btn = gr.Button("⚡ Ultra Fast (8 steps)", size="sm")
                        balanced_btn = gr.Button("⚖️ Balanced (15 steps)", size="sm")
                        quality_btn = gr.Button("🎨 High Quality (25 steps)", size="sm")

                with gr.Column(scale=1):
                    gr.Markdown("### 🖼️ Generated Image")
                    output_image = gr.Image(label="Generated Image", type="pil", height=400)
                    generation_status = gr.Textbox(label="📊 Generation Status", interactive=False)

                    # Time estimation display
                    gr.Markdown("### ⏱️ Expected Generation Times")
                    gr.HTML("""
                    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                                padding: 15px; border-radius: 10px; color: white;">
                        <b>Colab Performance (GPU):</b><br>
                        • 8 steps: ~10-15 seconds<br>
                        • 15 steps: ~20-30 seconds<br>
                        • 25 steps: ~40-60 seconds<br>
                        <small><i>512x512 resolution</i></small>
                    </div>
                    """)

        with gr.Tab("📊 System Info"):
            with gr.Column():
                gr.Markdown("### 💻 System Information")
                memory_info = gr.Textbox(label="Memory Usage", interactive=False)
                refresh_memory_btn = gr.Button("🔄 Refresh Memory Info")

                gr.Markdown("### ⚡ Optimization Tips")
                gr.HTML("""
                <div style="background: #f0f8ff; padding: 20px; border-radius: 10px; border-left: 4px solid #4CAF50;">
                    <h4>🚀 For Fastest Generation:</h4>
                    <ul>
                        <li><b>Use 8-15 inference steps</b> (sweet spot for speed/quality)</li>
                        <li><b>Keep resolution at 512x512</b> (perfect for most uses)</li>
                        <li><b>Use runwayml/stable-diffusion-v1-5</b> (fastest model)</li>
                        <li><b>Enable GPU in Colab</b> (Runtime → Change runtime type → GPU)</li>
                        <li><b>Clear memory between generations</b> (automatic in this interface)</li>
                    </ul>

                    <h4>🎨 For Best Quality:</h4>
                    <ul>
                        <li><b>Use detailed, descriptive prompts</b></li>
                        <li><b>Add style keywords:</b> "highly detailed", "cinematic", "4k"</li>
                        <li><b>Use negative prompts</b> to avoid unwanted elements</li>
                        <li><b>Guidance scale 7-8</b> works best for most images</li>
                    </ul>
                </div>
                """)

        # Event handlers
        load_btn.click(
            fn=generator.load_model,
            inputs=[model_dropdown],
            outputs=[load_status],
            show_progress=True
        )

        generate_btn.click(
            fn=generator.generate_image,
            inputs=[prompt, negative_prompt, steps, guidance, width, height, seed],
            outputs=[output_image, generation_status],
            show_progress=True
        )

        # Quick preset handlers
        def set_ultra_fast():
            return 8, 7.0, 512, 512

        def set_balanced():
            return 15, 7.5, 512, 512

        def set_high_quality():
            return 25, 8.0, 640, 640

        fast_btn.click(
            fn=set_ultra_fast,
            outputs=[steps, guidance, width, height]
        )

        balanced_btn.click(
            fn=set_balanced,
            outputs=[steps, guidance, width, height]
        )

        quality_btn.click(
            fn=set_high_quality,
            outputs=[steps, guidance, width, height]
        )

        refresh_memory_btn.click(
            fn=generator.get_memory_info,
            outputs=[memory_info]
        )

        # Load initial memory info
        interface.load(
            fn=generator.get_memory_info,
            outputs=[memory_info]
        )

    return interface

## 🎛️ `create_interface()` – Building the Gradio UI

**Purpose:**  
This function defines and returns my **Gradio interface**.  
It provides a **web-based GUI** for loading models, entering prompts, tuning settings, and generating images.  
I don’t interact directly with the class methods anymore — instead, I use buttons, sliders, and textboxes.

---

### 🔹 What happens step by step:

1. **Interface Setup**
   - I use `gr.Blocks` to build a flexible UI layout.
   - Title: `"Fast Stable Diffusion Generator"`.
   - Theme: `gr.themes.Soft()` → makes the UI visually cleaner.

---

2. **Header**
   - Adds a Markdown title and description:
     - `"# ⚡ Fast Stable Diffusion Text-to-Image Generator"`
     - `"Optimized for Google Colab with Real-time Progress Tracking"`

---

3. **🎨 Image Generation Tab**
   - Split into **two main columns**:
     - **Left Column (controls):**
       - **Model loading:**
         - Dropdown → select model version (`v1-5`, `2-1-base`, `v1-4`).
         - Load button → calls `generator.load_model(...)`.
         - Status textbox → shows loading results.
       - **Prompt settings:**
         - Prompt box (`✨ Prompt`) → main text input.
         - Negative prompt box (`🚫 Negative Prompt`) → optional filter for bad outputs.
       - **Generation parameters:**
         - Slider: `Inference Steps` (8–30).
         - Slider: `Guidance Scale` (3–15).
         - Dropdowns: `Width` + `Height` (256–768).
         - Seed: number input (default -1 for random).
       - **Generate button**:
         - `"🎨 Generate Image"` → runs `generator.generate_image(...)`.
       - **Quick Presets**:
         - Buttons for **Ultra Fast (8 steps)**, **Balanced (15 steps)**, **High Quality (25 steps)** → auto-adjust sliders.

     - **Right Column (outputs):**
       - `Generated Image` → shows the output.
       - `Generation Status` → shows progress messages.
       - **Expected generation times** (pre-calculated HTML info box).

---

4. **📊 System Info Tab**
   - Displays **system + optimization tips**:
     - `Memory Usage` box → updated with `generator.get_memory_info()`.
     - `Refresh` button → reloads memory info.
     - HTML panel → tips for speed vs quality:
       - Use **8–15 steps** for speed.
       - Keep resolution at **512×512** for efficiency.
       - Enable GPU in Colab.

---

5. **Event Handlers**
   - Connects UI elements to class functions:
     - `load_btn.click(...)` → calls `generator.load_model`.
     - `generate_btn.click(...)` → calls `generator.generate_image`.
     - Preset buttons (`fast_btn`, `balanced_btn`, `quality_btn`) → set sliders automatically.
     - `refresh_memory_btn.click(...)` → updates memory info.
     - `interface.load(...)` → shows memory usage immediately on startup.

---

### 🔹 In my code:
- This function **ties everything together**:
  - `generator.load_model` and `generator.generate_image` are now accessible via buttons.  
  - I can **switch models, generate images, and track GPU memory** from a clean UI.  
- When I call `create_interface()`, it returns the **Gradio app**, which I then launch with `.launch()` in Colab.

In [ ]:
if __name__ == "__main__":

    # Create and launch the interface
    interface = create_interface()

    # Launch with optimized settings for Colab
    interface.launch(
        share=True,  # Creates public link
        inbrowser=True,  # Opens in browser
        show_error=True,  # Show errors in interface
        quiet=False  # Show progress in console
    )